# Ural Federal University - Май 2022
## Межгалактический Хакатон 2022
Задача информационной безопасности: обнаружение атак на киберфизические системы<br>
Набор данных включает в себя состояние оборудования и характеристики очищаемой воды, а также сетевой трафик. Атаки в обучающем наборе данных размечены вручную. 
С помощью размеченных данных необходимо научиться определять атаки для тестового набора данных.<br>
________________________________________________________________________________
*Датасет Dataset to Support Research in the Design of Secure Water Treatment Systems (SWaT dataset)* -<br>
https://drive.google.com/u/0/uc?id=18BYm7zwvKvA4xdLsWgrWxNwYt4ttGcUJ&export=download&confirm=t<br>
Ссылка на GitHub c репозиторием - https://github.com/EkaterinaVZ/hackathon_2022
________________________________________________________________________________
## Часть 2/3

Dataset to Support Research in the Design of Secure Water Treatment Systems (SWaT dataset)


### Данный код выполняет следующие дейсвия:
<ul> <li>Для начала необходимо запустить часть 1, для создания файла ".csv с подготовленными и обработанными данными</li> 
<li>Считывает упомянутый выше файл в csv формате (пункт 1 списка)</li>
<li>Оценивает классификацую с использованием модели LogisticRegression()</li>
<li>Оценивает качество классификации с использованием 10 выбранных моделей обучения с параметрами по умолчанию</li>
<li>Представляет результаты с использованием scores</li>
<li>Оптимизирует гиперпараметры ML-алгоритмов и показывает результаты</li>
</ul>

### Things todo

<ul>
<li>Проверить другие инструменты для автоматического подбора гиперпараметров</li>
</ul>

###Log

<ul>
<li>Уменьшение размерности (с использованием PCA, не используем, в отдельном файле)</li>
<li>Настройка гиперпараметров с помощью GridSearchCV --> возможно, мы можем попробовать также случайный поиск CV или optuna</li>
</ul>
<li>Худшие (по скорости) классификаторы удалены из оценки</li>
<li>Настройка гиперпараметров завершена для выбранных моделей до 18.05.2022</li>

# ВСЕ МОДЕЛИ РАБОТАЮТ В СРЕДНЕМ С ACC НЕ МЕНЕЕ 96 -> Удовлетворены результатами

# 3 - Machine Learning Model Selection Analysis

Импорт основных библиотек

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=40e9f1ea7321e87bbb07c99f5f151a3d64c330d2c3a970c9eee835b9d6563ba1
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import wget
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

Собираем результаты работы классификатора в этот фрейм данных

In [66]:
model_scores = pd.DataFrame(columns=['Model', 'Accuracy','F1'])

## 3A - Data Import and Classifiers

In [4]:
# Score metrics etc.
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, plot_confusion_matrix
# ML models
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn import svm, tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate

import warnings
warnings.filterwarnings("ignore")

# Загружаем данные

In [11]:
data = pd.read_csv('/content/drive/MyDrive/Хакатон_2022/Datasets/FIX_Attack.csv', delimiter=';')

In [44]:
y = pd.DataFrame(data['A/N'])
y

,A/N
0,0
1,0
2,0
3,0
4,0
...,...
441793,0
441794,0
441795,0
441796,0


In [16]:
X = data.drop(columns='A/N')
X.shape

(441798, 44)

In [17]:
X

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT501,FIT502,FIT503,FIT504,P501,PIT501,PIT502,PIT503,FIT601,P602
0,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,1.723789,1.279621,0.735269,0.307786,2,250.8652,1.649953,189.5988,0.000128,1
1,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,1.723789,1.297554,0.735269,0.307786,2,250.8652,1.649953,189.6789,0.000128,1
2,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,1.723404,1.293967,0.735269,0.308619,2,250.8812,1.649953,189.6789,0.000128,1
3,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,1.723404,1.281158,0.735269,0.308619,2,250.8812,1.649953,189.6148,0.000128,1
4,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,2,...,1.723404,1.281158,0.735269,0.308619,2,250.8812,1.649953,189.5027,0.000128,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441793,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,1.716483,1.288971,0.734885,0.306569,2,251.1535,0.865024,189.0220,0.000000,1
441794,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,1.714560,1.272704,0.734885,0.306569,2,251.0734,0.865024,188.9259,0.000000,1
441795,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,1.714560,1.269502,0.734885,0.306569,2,251.0734,0.865024,188.9259,0.000000,1
441796,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,1.714560,1.263866,0.734885,0.307978,2,251.0734,0.865024,188.9259,0.000000,1


# Предварительная проверка гипотезы<br> с использованием модели LogisticRegression()


Разделение данных на обучающую и тестовую части 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42, stratify=y)

# Нормализируем данные

In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test) 

In [21]:
cat_columns = []
num_columns = []

for column_name in X.columns:
    if (X[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  [] 
 len =  0
numerical columns:	  ['FIT101', 'LIT101', ' MV101', 'P101', 'P102', ' AIT201', 'AIT202', 'AIT203', 'FIT201', ' MV201', ' P201', 'P203', ' P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', ' MV303', 'MV304', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P402', 'P403', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P602'] 
 len =  44


In [22]:
#@title Логистическая Регрессия{ run: "auto" }
#@markdown ### Тип Регуляризации 
penalty= 'l2' #@param [ "l2" , "l1", "none"]{type:"string"}
#@markdown ### Константа Регуларизации
regularization = 2.96 #@param {type:"slider", min:0.01, max:3, step:0.05}

scoring = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}


LogReg = LogisticRegression(fit_intercept=True,
                            penalty=penalty,solver='saga',
                            C=regularization,
                            max_iter=100000)

scores = cross_validate(LogReg, X_train, y_train,
                        scoring=scoring, 
                        cv=StratifiedKFold(n_splits=5, 
                        shuffle = True, 
                        random_state=42))

           
# Валидация на тренировочных данных
print('Результаты валидации')
DF_cv_logreg = pd.DataFrame(scores)
display(DF_cv_logreg)
print('\n')
print(DF_cv_logreg.mean()[2:])

Результаты валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,55.622100,0.129126,0.967584,0.848736,0.979283,0.748902
1,56.549290,0.127614,0.967293,0.846870,0.981404,0.744774
2,56.709228,0.127121,0.967632,0.849835,0.973200,0.754227
3,54.463012,0.135007,0.967729,0.849495,0.979312,0.750067
4,64.470093,0.127725,0.968004,0.851125,0.978381,0.753162




test_ACC          0.967648
test_F1           0.849212
test_Precision    0.978316
test_Recall       0.750226
dtype: float64


In [ ]:
B0=LogReg.intercept_[0]
B=LogReg.coef_[0]

features_names=X[num_columns].columns
# sign=['+','-']
# line='{:.2f}'.format(B0)
# for p,(fn,b) in enumerate(zip(features_names,B)):
#   if np.abs(b)>0:
#     line=line+sign[int(0.5*(np.sign(b)-1))]+'{:.2f}*'.format(np.abs(b))+fn

# print('Решение')
# print(line)
# weights_vis(B, B0, features_names)

# Валидация на тестовых данных

In [23]:
from sklearn.model_selection import cross_validate

scoring = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}
           
scores = cross_validate(LogReg, X_test, y_test,
                      scoring=scoring, cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42))


print('Результаты Кросс-валидации')
DF_cv_logreg = pd.DataFrame(scores)
display(DF_cv_logreg)
print('\n')
print(DF_cv_logreg.mean()[2:])

Результаты Кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,25.053514,0.057538,0.967519,0.849185,0.973103,0.753263
1,23.532083,0.052579,0.965973,0.839844,0.980108,0.734700
2,25.352768,0.053218,0.966614,0.843501,0.979064,0.740913
3,22.370749,0.052504,0.967595,0.847884,0.985997,0.743709
4,21.136184,0.054587,0.967067,0.846384,0.976055,0.747126




test_ACC          0.966953
test_F1           0.845360
test_Precision    0.978865
test_Recall       0.743942
dtype: float64


# Подбираем наилучшую модель для данного набора данных

## Создаем массив из классификаторов

Создаем более короткие имена классов для красивого и аккуратного вывода 

In [24]:
# NeuralNet, RandomForest, SVM  - очень долго проходят
classifiers = {
    "DecisionTree" : DecisionTreeClassifier(),        # max_depth=7, --> default criterion is 'gini'
    "NaiveBayes"   : GaussianNB(),
    "LogisticRegr" : LogisticRegression(),            # penalty='l1', solver='liblinear'
}
# classifiers = {
#     "kNN"          : KNeighborsClassifier(),          # n_neighbors=3, weights='uniform'
#     "DecisionTree" : DecisionTreeClassifier(),        # max_depth=7, --> default criterion is 'gini'
#     "RandomForest" : RandomForestClassifier(),        # max_depth=7, n_estimators=10, max_features=4
#     "NeuralNet"    : MLPClassifier(max_iter=500),     # alpha=1, max_iter parameter value decreased from 1000 to 500, no change, but faster.
#     "NaiveBayes"   : GaussianNB(),
#     "LDA"          : LinearDiscriminantAnalysis(),
#     "LDA(svd)"     : LinearDiscriminantAnalysis(solver='svd'), # Model must be optimized for 'svd' separately
#     "QDA"          : QuadraticDiscriminantAnalysis(),
#     "SVM"          : SVC(),                           # kernel="linear"          
#     "LogisticRegr" : LogisticRegression(),            # penalty='l1', solver='liblinear'
# }

In [25]:

# y['A/N'] = y['A/N'].map({0: 'Normal', 1: 'Attack'})
# y=pd.DataFrame(y['A/N'], dtype='category', columns=['A/N'])
# y

,A/N
0,Normal
1,Normal
2,Normal
3,Normal
4,Normal
...,...
441793,Normal
441794,Normal
441795,Normal
441796,Normal


In [46]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441798 entries, 0 to 441797
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   A/N     441798 non-null  int64
dtypes: int64(1)
memory usage: 3.4 MB


In [48]:
# Just in case, because you might wonder how to get corresponding codes and categories
cat_dict = {0: 'Normal', 1: 'Attack'}
# Following will give category names in the same order as they occur later in confusion matrixes
class_names = ['Normal', 'Attack']

['Normal', 'Attack']

In [49]:
y['A/N'].value_counts()

0    388150
1     53648
Name: A/N, dtype: int64

In [42]:
cat_dict

{0: 'Normal', 1: 'Attack'}

# Подгонка моделей к данным и проверка точности<br>
Скорость и качество

In [50]:
# Start timing the models above
from time import time
nfast = 14      # Don't run the very slow ones
head = list(classifiers.items())[:nfast]

for name, classifier in head:
    start = time()
    classifier.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    score = classifier.score(X_test, y_test)
    score_time = time()-start
    print("{:<15}| score = {:.3f} | time = {:,.3f}s/{:,.3f}s".format(name, score, train_time, score_time))

DecisionTree   | score = 0.999 | time = 14.052s/0.042s
NaiveBayes     | score = 0.961 | time = 0.222s/0.079s
LogisticRegr   | score = 0.967 | time = 7.743s/0.029s


Была проведена проверка на 10 моделях, это занимает очень мноо времени:

kNN            | score = 0.998 | time = 0.422s/911.650s<br>
DecisionTree   | score = 1.000 | time = 14.715s/0.351s<br>
RandomForest   | score = 1.000 | time = 111.327s/2.737s<br>
NeuralNet      | score = 0.991 | time = 2,052.556s/2.946s<br>
NaiveBayes     | score = 0.962 | time = 0.880s/0.407s<br>
LDA            | score = 0.965 | time = 3.970s/0.405s<br>
LDA(svd)       | score = 0.965 | time = 3.691s/0.414s<br>
QDA            | score = 0.967 | time = 1.978s/0.599s<br>
SVM            | score = 0.980 | time = 1,605.719s/268.465s<br>
LogisticRegr   | score = 0.968 | time = 8.635s/0.392s

## ОЦЕНКА ТОЧНОСТИ РАБОТЫ МОДЕЛЕЙ

In [67]:
for name, clf in list(classifiers.items()):
    # Fit data and make predictions
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # Accuracy and F1 scores
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    
    # Scores in dataframe
    scores = cross_validate(clf, X_train, y_train, scoring={'ACC': 'accuracy'},
                      cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))
    scores_df = pd.DataFrame(scores)
    # display(scores_df)
    
    # Сохранение разультатов
    score_row = {'Model':str(clf), 'Accuracy':acc, 'F1':f1}
    model_scores = model_scores.append(score_row, ignore_index=True)
    
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(data=cm, index=class_names, columns=class_names)
    

In [68]:
cm_df

,Normal,Attack
Normal,116215,231
Attack,4120,11974


In [69]:
model_scores.sort_values(by=['Accuracy'], ascending=False)

,Model,Accuracy,F1
0,DecisionTreeClassifier(),0.999313,0.999313
2,LogisticRegression(),0.967172,0.967172
1,GaussianNB(),0.961408,0.961408


## 3С - Подбор гиперпараметров с помощью GridSearchCV

In [70]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, ShuffleSplit
import scipy.stats as stats

In [71]:
model_opt_scores = pd.DataFrame(columns=['Algorithm','best_params','best_accuracy','f1_macro','f1_micro'])

Проверка доступных моделей

In [72]:
classifiers

{'DecisionTree': DecisionTreeClassifier(),
 'LogisticRegr': LogisticRegression(),
 'NaiveBayes': GaussianNB()}

In [73]:
clfs = [classifiers.get('DecisionTree'), classifiers.get('LogisticRegr'), classifiers.get('NaiveBayes')]
clfs

[DecisionTreeClassifier(), LogisticRegression(), GaussianNB()]

## Установиливаем настраиваемые параметры для всех ML-алгоритмов

In [74]:
# Можем проверить доступные гиперпараметры для каждой из моделей
classifiers.get('DecisionTree').get_params().keys()

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [75]:
params = {
    'DecisionTree': {'criterion':['gini','entropy'],
                     'max_depth': np.arange(3, 15),
                     'splitter': ['best','random']}, 
    'NaiveBayes'  : {'var_smoothing': np.logspace(0,-9, num=100)}, 
    'LogisticReg' : {'C': np.logspace(-4, 4, 50),
                     'penalty': ['l1', 'l2']}
}

**Попытка оптимизировать с помощью GridSearch параметры в цикле**

In [76]:
for clf, p in zip(classifiers.values(), params.values()):
    print(' ', '\033[1m➥' + str(clf) + '\033[0m') # Печать текста жирным шрифтом
    clf_search = GridSearchCV(estimator=clf,
                              refit=True,
                              verbose=1,
                              param_grid=p,
                              cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))
    clf_search.fit(X_train, y_train)
    clf_best = clf_search.best_estimator_
    clf_best.fit(X_train, y_train)
    y_pred = clf_best.predict(X_test)
    
    # test error
    score_row = {'Algorithm': str(clf),
                 'best_params':clf_search.best_params_,
                 'best_accuracy': accuracy_score(y_pred, y_test),
                 'f1_macro': f1_score(y_pred, y_test, average='micro'),
                 'f1_micro': f1_score(y_pred, y_test, average='macro')}
    model_opt_scores = model_opt_scores.append(score_row, ignore_index=True)
    print(classification_report(y_test, y_pred, target_names=class_names))
print('Оптимизация выполнена!')

  ➥DecisionTreeClassifier()
Fitting 5 folds for each of 48 candidates, totalling 240 fits
              precision    recall  f1-score   support

      Normal       0.99      1.00      1.00    116446
      Attack       0.99      0.95      0.97     16094

    accuracy                           0.99    132540
   macro avg       0.99      0.97      0.98    132540
weighted avg       0.99      0.99      0.99    132540

  ➥GaussianNB()
Fitting 5 folds for each of 100 candidates, totalling 500 fits
              precision    recall  f1-score   support

      Normal       0.96      1.00      0.98    116446
      Attack       0.98      0.70      0.82     16094

    accuracy                           0.96    132540
   macro avg       0.97      0.85      0.90    132540
weighted avg       0.96      0.96      0.96    132540

  ➥LogisticRegression()
Fitting 5 folds for each of 100 candidates, totalling 500 fits
              precision    recall  f1-score   support

      Normal       0.97      1.00  

# Анализ результатов

In [77]:
pd.set_option("display.max_colwidth", None)
model_opt_scores.sort_values(by=['best_accuracy'], ascending=False)

,Algorithm,best_params,best_accuracy,f1_macro,f1_micro
0,DecisionTreeClassifier(),"{'criterion': 'entropy', 'max_depth': 14, 'splitter': 'best'}",0.992485,0.992485,0.982114
2,LogisticRegression(),"{'C': 339.3221771895323, 'penalty': 'l2'}",0.967813,0.967813,0.915945
1,GaussianNB(),{'var_smoothing': 0.005336699231206307},0.961634,0.961634,0.897407


In [78]:
# настроим формат таблиц в pandas
pd.set_option("display.max_colwidth", 70)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)
comp_results = pd.concat([model_scores.iloc[:len(classifiers),:], model_opt_scores.iloc[:,1:]], axis=1)
comp_results.sort_values(by='best_accuracy', ascending=False)

,Model,Accuracy,F1,best_params,best_accuracy,f1_macro,f1_micro
0,DecisionTreeClassifier(),0.999,0.999,"{'criterion': 'entropy', 'max_depth': 14, 'splitter': 'best'}",0.992,0.992,0.982
2,LogisticRegression(),0.967,0.967,"{'C': 339.3221771895323, 'penalty': 'l2'}",0.968,0.968,0.916
1,GaussianNB(),0.961,0.961,{'var_smoothing': 0.005336699231206307},0.962,0.962,0.897
